In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Initialize Spotify API credentials
client_id = '50b05cd601fb450786893623c7faa8e3'
client_secret = '76944756995946558edc222379e88eef'

# Authenticate
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Function to get tracks from a popular playlist (e.g., Top 50 Global)
def get_top_tracks_from_playlist(playlist_id, limit=50):
    results = sp.playlist_tracks(playlist_id, limit=limit)
    return results['items']

# Fetch tracks from the "Top 50 - Global" playlist
playlist_id = '37i9dQZEVXbMDoHDwVN2tF'  # Spotify ID for "Top 50 - Global" playlist
top_tracks = get_top_tracks_from_playlist(playlist_id)

# Extract track IDs and other required metadata
track_data = []
for item in top_tracks:
    track = item['track']
    track_info = {
        'track_id': track['id'],
        'artists': ', '.join([artist['name'] for artist in track['artists']]),
        'album_name': track['album']['name'],
        'track_name': track['name'],
        'popularity': track['popularity'],
        'duration_ms': track['duration_ms'],
        'explicit': track['explicit'],
        'track_genre': ', '.join(sp.artist(track['artists'][0]['id'])['genres']) if sp.artist(track['artists'][0]['id'])['genres'] else 'Unknown'
    }
    track_data.append(track_info)

# Get audio features for the top tracks
track_ids = [track['track_id'] for track in track_data]
audio_features = sp.audio_features(track_ids)

# Combine metadata and audio features
for i, features in enumerate(audio_features):
    if features:
        track_data[i].update({
            'danceability': features['danceability'],
            'energy': features['energy'],
            'key': features['key'],
            'loudness': features['loudness'],
            'mode': features['mode'],
            'speechiness': features['speechiness'],
            'acousticness': features['acousticness'],
            'instrumentalness': features['instrumentalness'],
            'liveness': features['liveness'],
            'valence': features['valence'],
            'tempo': features['tempo'],
            'time_signature': features['time_signature']
        })

# Convert to DataFrame for better readability and manipulation
df = pd.DataFrame(track_data)

# Ensure the columns are in the specified order
column_order = ['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']
df = df[column_order]

# Save to CSV in the current working directory
csv_file_path = 'spotify_tracks.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")


Data saved to spotify_tracks.csv
